In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import cv2

sys.path.append('util')
from camera_utils import process_camera_json, world_to_pixel


In [50]:
intrin, extrin, extrin_inv = process_camera_json('content2/slahmr/outputs/logs/video-val/2024-03-10/shorto-all-shot-0-0-180/smooth_fit/shorto_cameras_000060.json')

In [52]:
print(intrin)

print(extrin)

[[1.5e+03 0.0e+00 9.6e+02]
 [0.0e+00 1.5e+03 5.4e+02]
 [0.0e+00 0.0e+00 1.0e+00]]
None


In [32]:
smoothed_file = 'content2/slahmr/outputs/logs/video-val/2024-03-10/shorto-all-shot-0-0-180/smooth_fit/shorto_000060_world_results.npz'
smoothed_data = np.load(smoothed_file)
transformations = smoothed_data['trans']
root_orient = smoothed_data['root_orient']

# Ped 0 step 0
ped_0_poses = transformations[1,:,:]
print(ped_0_poses.shape)
print(ped_0_poses[100])
# Convert world to pixel
intrin = np.array(intrin)
u_v = intrin @ ped_0_poses[100].T
u_v = u_v / u_v[2]
u_v = u_v[:2]
print(u_v)

# get 


(150, 3)
[ 3.84636    -0.82861257 12.2781515 ]
[1429.90297738  438.76987202]


In [55]:
transformations = smoothed_data['trans']

# Make uv empty array to store pixel coordinates
# should produce a (N,T,2) array

uv_transformations = np.zeros((transformations.shape[0], transformations.shape[1], 2))

# get u_v for (N,T,3) transformations array

for n in range(transformations.shape[0]):
    for t in range(transformations.shape[1]):
        u_v = intrin @ transformations[n,t].T
        u_v = u_v / np.abs(u_v[2])
        u_v = u_v[:2]

        if u_v[0] < 0 or u_v[0] > 1920 or u_v[1] < 0 or u_v[1] > 1080:
            print(n,t)
        #     u_v = np.array([-1,-1])

        uv_transformations[n,t] = u_v

        

print(uv_transformations.shape)

def pixel_to_3d_point(u, v, Z, fx, fy, cx, cy):
    """
    Convert a pixel coordinate (u, v) with a given depth Z to a 3D point in camera coordinates.

    Parameters:
    - u, v: Pixel coordinates in the image.
    - Z: Depth value at the pixel (u, v).
    - fx, fy: Focal lengths of the camera along the x and y axes.
    - cx, cy: Coordinates of the principal point (typically the image center).

    Returns:
    - A tuple (X, Y, Z) representing the 3D point in camera coordinates.
    """
    # Normalize pixel coordinates
    x = (u - cx) / fx
    y = (v - cy) / fy

    # Apply depth to get 3D point
    X = x * Z
    Y = y * Z

    return (X, Y, Z)



# Save the transformations and uv_transformations to a new npz file
np.savez('shorto_000060_pixel_results.npz', uv_transformations=uv_transformations, transformations=transformations)




6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
10 133
10 134
10 135
10 136
10 137
10 138
10 139
10 140
10 141
10 142
10 143
10 144
10 145
10 146
10 147
10 148
10 149
11 132
11 133
11 134
11 135
11 136
11 137
11 138
11 139
11 140
11 141
11 142
11 143
11 144
11 145
11 146
11 147
11 148
11 149
(12, 150, 2)


In [54]:
uv_transformations

array([[[  877.44273748,   326.76543872],
        [  875.50791799,   327.73540739],
        [  873.01066407,   329.92618062],
        ...,
        [  299.74278492,   633.12114362],
        [  293.8320895 ,   634.52657492],
        [  289.43815459,   635.2934689 ]],

       [[  908.88895033,   578.11658754],
        [  914.23526949,   575.14372349],
        [  920.61153113,   573.25003464],
        ...,
        [ 1580.44493608,   365.6896288 ],
        [ 1582.28244988,   364.62214875],
        [ 1583.63094548,   364.14309268]],

       [[ 1316.33028323,   308.45034609],
        [ 1315.02632273,   308.66994423],
        [ 1313.62767533,   309.42425587],
        ...,
        [  959.42996929,   591.37338396],
        [  957.96423259,   593.536142  ],
        [  956.63551577,   594.95339146]],

       ...,

       [[ -117.85728782, -1385.21539891],
        [ -112.93892865, -1427.74967755],
        [ -130.62163157, -1428.97959812],
        ...,
        [ 1029.19095991,   178.75679052],
     

In [42]:
# read the npz file
data = np.load('shorto_000060_pixel_results.npz')
print(data['uv_transformations'].shape)

(12, 150, 2)


In [27]:
import imageio

# Initialize video capture
video = 'content2/slahmr/outputs/logs/video-val/2024-03-10/shorto-all-shot-0-0-180/shorto_input_final_000000_src_cam.mp4'  # Update this to the correct path
cap = cv2.VideoCapture(video)
if not cap.isOpened():
    raise IOError("Error opening video stream or file")

# Prepare for GIF creation
frames_for_gif = []

# Process each frame
frame_idx = 0
for frame_idx in range(100):

    # Get ped_0's pose for the current frame in world coordinates
    ped_0_world_pose = transformations[1][frame_idx]
    
    # Convert world coordinates to pixel coordinates
    ped_0_pixel_pose = world_to_pixel(ped_0_world_pose, intrin)
    
    print(f"Frame {frame_idx}: {ped_0_pixel_pose}")

    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break


    # Draw the pose on the frame
    cv2.circle(frame, tuple(ped_0_pixel_pose.astype(int)), 5, (0, 0, 255), -1)

    # Append the frame to the list of frames for the GIF
    frames_for_gif.append(frame)

# Save the GIF
gif_path = 'ped_0_pose.gif'
imageio.mimsave(gif_path, frames_for_gif, fps=30)
print(f"Saved GIF to {gif_path}")




Frame 0: [908.88895033 578.11658754]
Frame 1: [914.23526949 575.14372349]
Frame 2: [920.61153113 573.25003464]
Frame 3: [926.90911065 571.56424957]
Frame 4: [932.66265833 570.0687749 ]
Frame 5: [938.50629425 569.4820293 ]
Frame 6: [945.07149115 569.59231034]
Frame 7: [951.36553901 570.50738763]
Frame 8: [957.44474139 572.02264297]
Frame 9: [963.02000341 572.64674783]
Frame 10: [969.01092438 571.74990719]
Frame 11: [975.54815855 570.43941775]
Frame 12: [982.35281056 568.52012323]
Frame 13: [988.64282406 566.16235224]
Frame 14: [994.95800694 564.451098  ]
Frame 15: [1001.06039942  562.49090564]
Frame 16: [1006.51900518  560.98563402]
Frame 17: [1012.1557164   559.66306127]
Frame 18: [1017.65152978  559.01731322]
Frame 19: [1023.85164333  558.64295127]
Frame 20: [1030.28382705  558.45913325]
Frame 21: [1036.80028342  558.59708524]
Frame 22: [1043.29806975  557.99667796]
Frame 23: [1049.80466873  556.92696431]
Frame 24: [1057.29768608  555.2948672 ]
Frame 25: [1064.87942341  553.69260438]


In [28]:
import numpy as np 

def get_intrinsics(H,W):
    """
    Intrinsics for a pinhole camera model.
    Assume fov of 55 degrees and central principal point.
    """
    f = 0.5 * W / np.tan(0.5 * 90 * np.pi / 180.0)
    cx = 0.5 * W
    cy = 0.5 * H
    return np.array([[f, 0, cx],
                     [0, f, cy],
                     [0, 0, 1]])

def depth_to_points(depth, R=None, t=None):

    K = get_intrinsics(depth.shape[0], depth.shape[1])
    Kinv = np.linalg.inv(K)
    if R is None:
        R = np.eye(3)
    if t is None:
        t = np.zeros(3)

    # M converts from your coordinate to PyTorch3D's coordinate system
    M = np.eye(3)
    M[0, 0] = -1.0
    M[1, 1] = -1.0

    height, width = depth.shape[0], depth.shape[1]

    x = np.arange(width)
    y = np.arange(height)
    coord = np.stack(np.meshgrid(x, y), -1)
    coord = np.concatenate((coord, np.ones_like(coord)[:, :, [0]]), -1)  # z=1
    coord = coord.astype(np.float32)
    # coord = torch.as_tensor(coord, dtype=torch.float32, device=device)
    coord = coord[None]  # bs, h, w, 3
    # convert depth from hxw to 1xhxw
    depth_convert = depth[None, :, :]

    D = depth_convert[:, :, :, None, None]
    # print(D.shape, Kinv[None, None, None, ...].shape, coord[:, :, :, :, None].shape )
    pts3D_1 = D * Kinv[None, None, None, ...] @ coord[:, :, :, :, None]
    # pts3D_1 live in your coordinate system. Convert them to Py3D's
    pts3D_1 = M[None, None, None, ...] @ pts3D_1
    # from reference to targe tviewpoint
    pts3D_2 = R[None, None, None, ...] @ pts3D_1 + t[None, None, None, :, None]
    # pts3D_2 = pts3D_1
    # depth_2 = pts3D_2[:, :, :, 2, :]  # b,1,h,w
    return pts3D_2[:, :, :, :3, 0][0]


def create_triangles(h, w, mask=None):
    """
    Reference: https://github.com/google-research/google-research/blob/e96197de06613f1b027d20328e06d69829fa5a89/infinite_nature/render_utils.py#L68
    Creates mesh triangle indices from a given pixel grid size.
        This function is not and need not be differentiable as triangle indices are
        fixed.
    Args:
    h: (int) denoting the height of the image.
    w: (int) denoting the width of the image.
    Returns:
    triangles: 2D numpy array of indices (int) with shape (2(W-1)(H-1) x 3)
    """
    x, y = np.meshgrid(range(w - 1), range(h - 1))
    tl = y * w + x
    tr = y * w + x + 1
    bl = (y + 1) * w + x
    br = (y + 1) * w + x + 1
    triangles = np.array([tl, bl, tr, br, tr, bl])
    triangles = np.transpose(triangles, (1, 2, 0)).reshape(
        ((w - 1) * (h - 1) * 2, 3))
    if mask is not None:
        mask = mask.reshape(-1)
        triangles = triangles[mask[triangles].all(1)]
    return triangles

In [29]:
import numpy as np
import trimesh
from functools import partial
import os
import sys


def depth_edges_mask(depth):
    """Returns a mask of edges in the depth map.
    Args:
    depth: 2D numpy array of shape (H, W) with dtype float32.
    Returns:
    mask: 2D numpy array of shape (H, W) with dtype bool.
    """
    # Compute the x and y gradients of the depth map.
    depth_dx, depth_dy = np.gradient(depth)
    # Compute the gradient magnitude.
    depth_grad = np.sqrt(depth_dx ** 2 + depth_dy ** 2)
    # Compute the edge mask.
    mask = depth_grad > 0.05
    return mask

def get_mesh(depth, image, keep_edges=False):
    # image.thumbnail((1024,1024))  # limit the size of the input image
    pts3d = depth_to_points(depth)
    pts3d = pts3d.reshape(-1, 3)

    # Create a trimesh mesh from the points
    # Each pixel is connected to its 4 neighbors
    # colors are the RGB values of the image

    verts = pts3d.reshape(-1, 3)
    image = np.array(image)
    if keep_edges:
        triangles = create_triangles(image.shape[0], image.shape[1])
    else:
        triangles = create_triangles(image.shape[0], image.shape[1], mask=~depth_edges_mask(depth))
    colors = image.reshape(-1, 3)
    mesh = trimesh.Trimesh(vertices=verts, faces=triangles, vertex_colors=colors)

    # Save as glb in folder
    glb_file = "mesh.glb"
    mesh.export(glb_file)
    glb_path = os.path.abspath(glb_file)


    return glb_path


glb_path = get_mesh(np.array(depth), imageRGB, keep_edges=True)
print(glb_path)

/Users/omoruyiatekha/Documents/GitHub/pedTrack/mesh.glb


In [23]:
print(np.array(image).shape)

# convert image to C x H x W
imageC = np.array(image).transpose(2, 0, 1)

# convert imageC which has an alpha channel to RGB
imageRGB = imageC[:3]
imageRGB = imageRGB.transpose(1, 2, 0)
imageRGB.shape


(1972, 3494, 4)


(1972, 3494, 3)

In [10]:
np.array(depth).shape[0]

1972

In [30]:
# Use OpenCV to reconstruct the 3D point cloud with the depth map and color image
import cv2
import numpy as np
import open3d as o3d

# convert depth to numpy array
depth = np.array(depth)

def get_intrinsics(H,W):
    """
    Intrinsics for a pinhole camera model.
    Assume fov of 55 degrees and central principal point.
    """
    f = 0.5 * W / np.tan(0.5 * 55 * np.pi / 180.0)
    cx = 0.5 * W
    cy = 0.5 * H
    return np.array([[f, 0, cx],
                     [0, f, cy],
                     [0, 0, 1]])


disparity = depth

intrinsics = get_intrinsics(*depth.shape)
Q = np.array([[1, 0, 0, -0.5 * depth.shape[1]],
              [0, -1, 0, 0.5 * depth.shape[0]],
              [0, 0, 0, -intrinsics[0, 0]],
              [0, 0, 1, 0]])

points = cv2.reprojectImageTo3D(disparity, Q)
colors = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)

# Create an Open3D point cloud object
pcd = o3d.geometry.PointCloud()

# Assign points and colors to the point cloud object
pcd.points = o3d.utility.Vector3dVector(points.reshape(-1, 3))  # Flatten the points array
pcd.colors = o3d.utility.Vector3dVector(colors.reshape(-1, 3) / 255.0)  # Flatten and normalize the colors array

# Estimate normals
pcd.estimate_normals()

# make point cloud
o3d.visualization.draw_geometries([pcd])

# make mesh
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)


